In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp drive/MyDrive/data-mining/* ./

In [ ]:
INOUT = pd.read_csv('INOUT.csv')
INOUT.name = "INOUT"

INOUTLINE = pd.read_csv('INOUTLINE.csv')
INOUTLINE.name = "INOUTLINE"

PRODUCTS = pd.read_csv('PRODUCTS.csv')
PRODUCTS.name = "PRODUCTS"

TRANSFER_ITEM = pd.read_csv('TRANSFER_ITEM.csv')
TRANSFER_ITEM.name = "TRANSFER_ITEM"

TRANSFER_ITEM_D = pd.read_csv('TRANSFER_ITEM_D.csv')
TRANSFER_ITEM_D.name = "TRANSFER_ITEM_D"

PRODUCTINSTANCE = pd.read_csv('PRODUCTINSTANCE.csv', encoding='cp1258', low_memory=False)
PRODUCTINSTANCE.name = "PRODUCTINSTANCE"

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,12,32,39,40,41,44,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8,26,68,69,70,84,88) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,12,13,14,15,16,17,18,24,26,29) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


#Utils

In [ ]:
def countRecords(df):
   return df.shape[0]


In [ ]:
def countNull(df):
  return df.isnull().sum()

In [ ]:
def countNotNull(df, col_name):
  total_count = countRecords(df)
  null_count = int(countNull(df[col_name].to_frame()))
  count = total_count - null_count

  return count

In [ ]:
def completeness(df):
  count = countRecords(df)
  result = count - countNull(df)
  return (result / count) * 100

In [ ]:
def validity(df, col_name, possible_values):
  column = df.loc[:,col_name]

  # count the number of valid values
  valid = 0
  for i in range(len(column)):
    if column[i] in possible_values:
      valid += 1

  result = (valid / countNotNull(df, col_name)) * 100
  print(result)

In [ ]:
def validityInRange(df, col_name, numeric_range):
  column = df.loc[:,col_name]
  valid = 0

  for i in range(len(column)):
    if column[i] != 'null':
      if column[i] >= numeric_range[0] and column[i] <= numeric_range[1]:
        valid += 1

  result = (valid / countNotNull(df, col_name)) * 100
  print(result)

#Question 2

##single source

###schema

####primary key

In [ ]:
INOUT.loc[:,'INOUT_ID'].is_unique

True

In [ ]:
INOUTLINE.loc[:,'INOUTLINE_ID'].is_unique

True

In [ ]:
TRANSFER_ITEM.loc[:,'TRANSFER_ITEM_ID'].is_unique

True

In [ ]:
TRANSFER_ITEM_D.loc[:,'TRANSFER_ITEM_D_ID'].is_unique

False

In [ ]:
PRODUCTS.loc[:,'M_PRODUCT_ID'].is_unique

True

In [ ]:
PRODUCTINSTANCE.loc[:,'PRODUCTINSTANCE_ID'].is_unique

False

####foreign key

In [ ]:
keys = ['INOUT_ID','INOUTLINE_ID','TRANSFER_ITEM_ID','TRANSFER_ITEM_D_ID','M_PRODUCT_ID','PRODUCTINSTANCE_ID']

prm_keys_dictionary = {
  'INOUT_ID': INOUT,
  'INOUTLINE_ID': INOUTLINE,
  'TRANSFER_ITEM_ID': TRANSFER_ITEM,
  'TRANSFER_ITEM_D_ID': TRANSFER_ITEM_D,
  'M_PRODUCT_ID': PRODUCTS,
  'PRODUCTINSTANCE_ID': PRODUCTINSTANCE
}

In [ ]:
def check_equality(serie1, serie2):
  try:
    cp_s1 = serie1.copy()
    cp_s1.sort_values(inplace=True)

    cp_s2 = serie2.copy()
    cp_s2.sort_values(inplace=True)

    return np.array_equal(cp_s1, cp_s2, equal_nan=False)
  except (TypeError,ValueError):
    return False

In [ ]:
def find_fkey(df,prm_key):
  fkeys = keys.copy()
  fkeys.remove(prm_key)
  column_headers = list(df.columns.values)

  for i in column_headers:
    if i in fkeys:
      table = prm_keys_dictionary[i]
      if not check_equality(table[i], df[i]):
        print(f'records of {i} in its table "{table.name}" are not equal with the table "{df.name}"')
      else:
        print('records are the same in both tables')

In [ ]:
find_fkey(INOUT,'INOUT_ID')

records of M_PRODUCT_ID in its table "PRODUCTS" are not equal with the table "INOUT"


In [ ]:
find_fkey(INOUTLINE,'INOUTLINE_ID')

records of INOUT_ID in its table "INOUT" are not equal with the table "INOUTLINE"
records of M_PRODUCT_ID in its table "PRODUCTS" are not equal with the table "INOUTLINE"


In [ ]:
find_fkey(TRANSFER_ITEM_D,'TRANSFER_ITEM_D_ID')

records of TRANSFER_ITEM_ID in its table "TRANSFER_ITEM" are not equal with the table "TRANSFER_ITEM_D"
records of M_PRODUCT_ID in its table "PRODUCTS" are not equal with the table "TRANSFER_ITEM_D"


In [ ]:
find_fkey(PRODUCTINSTANCE,'PRODUCTINSTANCE_ID')

records of M_PRODUCT_ID in its table "PRODUCTS" are not equal with the table "PRODUCTINSTANCE"


In [ ]:
find_fkey(TRANSFER_ITEM,'TRANSFER_ITEM_ID')

In [ ]:
find_fkey(PRODUCTS,'M_PRODUCT_ID')

####data in domain range

In [ ]:
validity(INOUT,"C_DOCSTATUS_ID",[3000025,3000006,3000018])
validity(INOUT,"ACCEPTANCETYPE",['T','S','H'])
validity(INOUT,"ISACTIVE",['Y','N','n','y'])

99.97368421052632
100.0
100.0


In [ ]:
validity(PRODUCTINSTANCE,"ISACTIVE",['Y','N','n','y'])
validity(PRODUCTINSTANCE,"ISCONTROLAMVAL",['Y','N','n','y'])

validityInRange(PRODUCTINSTANCE, 'RETURNAMVALTOANBAR', [1,4])

99.93419859066915
43.37922007582596
100.0


In [ ]:
validity(TRANSFER_ITEM,"ISACTIVE",['Y','N','n','y'])
validity(TRANSFER_ITEM,"ISCONTROL",['Y','N','n','y'])

99.97650191900995
100.0


In [ ]:
validity(TRANSFER_ITEM_D,"ISACTIVE",['Y','N','n','y'])

99.99194257743518


###instance

####missing values

In [ ]:
def has_missing_value(df):
  complete_list = completeness(df)
  for i in range(len(complete_list)):
    if complete_list[i] == 100.000000:
      return f"there is no missing value in {df.name}"
    else:
      return f"there exist missing values in {df.name}"

In [ ]:
has_missing_value(INOUT)

'there is no missing value in INOUT'

In [ ]:
has_missing_value(INOUTLINE)

'there is no missing value in INOUTLINE'

In [ ]:
has_missing_value(PRODUCTS)

'there is no missing value in PRODUCTS'

In [ ]:
has_missing_value(TRANSFER_ITEM)

'there is no missing value in TRANSFER_ITEM'

In [ ]:
has_missing_value(TRANSFER_ITEM_D)

'there exist missing values in TRANSFER_ITEM_D'

In [ ]:
has_missing_value(PRODUCTINSTANCE)

'there exist missing values in PRODUCTINSTANCE'

####duplicated records

In [ ]:
def has_duplicated_value(df):
  duplicated_list = df.duplicated(keep=False)
  for i in range(len(duplicated_list)):
    if duplicated_list[i] == True:
      return f"there exits duplicated record in {df.name}"

  return f"there is no duplicated record in {df.name}"

In [ ]:
has_duplicated_value(TRANSFER_ITEM_D)

'there is no duplicated record in TRANSFER_ITEM_D'

In [ ]:
has_duplicated_value(PRODUCTINSTANCE)

'there exits duplicated record in PRODUCTINSTANCE'

##multi source

###schema

####violated attribute dependencies

In [ ]:
REACHMENTDATE = TRANSFER_ITEM_D.loc[:,'REACHMENTDATE']
OPERATIONDATE = TRANSFER_ITEM.loc[:,'OPERATIONDATE']

TRANSFER_ITEM_D_ID = TRANSFER_ITEM_D.loc[:,'TRANSFER_ITEM_D_ID']
TRANSFER_ITEM_ID = TRANSFER_ITEM.loc[:,'TRANSFER_ITEM_ID']

In [ ]:
from datetime import datetime

In [ ]:
for i in range(len(TRANSFER_ITEM_D_ID)):
  for j in range(len(TRANSFER_ITEM_ID)):
    if TRANSFER_ITEM_D_ID[i] == TRANSFER_ITEM_ID[j]:
      r = pd.to_datetime(REACHMENTDATE[i], format='%m/%d/%Y %I:%M:%S %p')
      o = datetime.strptime(OPERATIONDATE[j], '%m/%d/%Y %I:%M:%S %p')
      if r > o:
        print('there is an violated attribute dependencies in REACHMENTDATE and OPERATIONDATE')

KeyboardInterrupt: ignored

###instance

####inconsistent aggregation

In [ ]:
validity(PRODUCTS,"ISSUMMARY",['Y','N','n','y'])


100.0


In [ ]:
validity(PRODUCTINSTANCE,"ISSUMMARY",['Y','N','n','y'])

99.67059624133591


####duplicated records

In [ ]:
def merge_dfs(df1, df2):
  T = df1.copy()
  D = df2.copy()
  r = pd.concat([T,D], axis=1)
  r = r.reindex(T.index)
  r.name = f'merged of "{df1.name} and {df2.name}"'

  return r

In [ ]:
has_duplicated_value(merge_dfs(TRANSFER_ITEM,TRANSFER_ITEM_D))

'there is no duplicated record in merged of "TRANSFER_ITEM and TRANSFER_ITEM_D"'

In [ ]:
has_duplicated_value(merge_dfs(PRODUCTS,PRODUCTINSTANCE))

'there is no duplicated record in merged of "PRODUCTS and PRODUCTINSTANCE"'

In [ ]:
has_duplicated_value(merge_dfs(PRODUCTS,INOUT))

'there is no duplicated record in merged of "PRODUCTS and INOUT"'

In [ ]:
has_duplicated_value(merge_dfs(INOUTLINE,INOUT))

'there is no duplicated record in merged of "INOUTLINE and INOUT"'

In [ ]:
has_duplicated_value(merge_dfs(PRODUCTS,TRANSFER_ITEM_D))

'there is no duplicated record in merged of "PRODUCTS and TRANSFER_ITEM_D"'